# Coordinates

This file contains the processing of coordinates data in the yelp restaurant dataset.

**Goal:** Provide a coordinates_processing function, which can be used in main.ipynb.

## 1. Get the coordinates dataset

The coordinates dataset only contains price, rating, review_count, name, id, coordinates (latitude, longitude) columns. Extraction of this dataset is shown in main.ipynb.

In [1]:
import pickle
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt

In [2]:
data_file = '../Dataset/coordinates_data'

with open(data_file, 'rb') as inFile:
    data = pickle.load(inFile)

display(data.head())
print("data.shape: {0}".format(data.shape))

,id,name,latitude,longitude,rating,review_count,price
0,bwCj2AcoOroZfCTxb6rCcg,A Better Burger,36.689639,-83.108766,3.5,6,2
1,S9S9kFJSkmfpbjFForCWLQ,El Castillo,36.726337,-83.099858,4.0,2,1
2,np8uV1xll22Yr-Q-B-ImkA,Rooster's Pub,36.758436,-83.027057,4.5,4,1
3,HGY1ojoLu07P_ky2LeRguQ,Redstone Restaurant,36.689259,-82.753040,4.5,3,1
4,J5XS3VmxnLKhNlpiwDJ-3A,Little Mexico,36.859367,-82.756744,4.0,5,1


data.shape: (5119, 7)


## 2. Generate dataset for Gephi

Now we are going to consider the relation between restaurants as a undirected network graph. Definition of this graph is listed as follows:

**Node:** Each node represents a business, which can be uniquely represented by its id.

**Edge:** Each edge $(u, v)$ represents that the business $u$ is a neighbor of $v$ $(u \neq v)$. Here, we define a parameter, neighbor_dist, as the maximum distance (kilometer) between two neighbors. If the distance of two businesses, $u$ and $v$, is less than neighbor_dist, then we will have edge $(u, v)$ and $(v, u)$.

To calculate the distance between two businesses on the earth, we here adopted the Haversine formula. The code is based on [Haversine formula in Python](https://stackoverflow.com/a/4913653/8163369) on StackOverFlow.

In [3]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [22]:
neighbor_dist = 0.5 # The maximum distance (kilometer) between two neighbors

lats, lons = np.array(data['latitude']), np.array(data['longitude']) # latitude and longitude of businesses
num_samples = data.shape[0] # total number of businesses

edges = []

for i in range(0, num_samples):
    for j in range(i + 1, num_samples):
        dist = haversine(lons[i], lats[i], lons[j], lats[j])
        if(dist < neighbor_dist):
            edges.append([i, j, dist + 0.001])
            edges.append([j, i, dist + 0.001])

In [23]:
len(edges)

21978

**Output nodes and as .csv file:** for use of Gephi.

In [24]:
# Generate edges and nodes dataframe as the requirement of Gephi
edges_df = pd.DataFrame(edges, columns=['Source', 'Target', 'Weight'])

nodes_df = data.rename(columns={'id': 'label'})
nodes_df = nodes_df.assign(id=range(0, num_samples))

display(edges_df.head())
display(nodes_df.head())

,Source,Target,Weight
0,12,13,0.181609
1,13,12,0.181609
2,14,17,0.284748
3,17,14,0.284748
4,14,18,0.172542


,label,name,latitude,longitude,rating,review_count,price,id
0,bwCj2AcoOroZfCTxb6rCcg,A Better Burger,36.689639,-83.108766,3.5,6,2,0
1,S9S9kFJSkmfpbjFForCWLQ,El Castillo,36.726337,-83.099858,4.0,2,1,1
2,np8uV1xll22Yr-Q-B-ImkA,Rooster's Pub,36.758436,-83.027057,4.5,4,1,2
3,HGY1ojoLu07P_ky2LeRguQ,Redstone Restaurant,36.689259,-82.753040,4.5,3,1,3
4,J5XS3VmxnLKhNlpiwDJ-3A,Little Mexico,36.859367,-82.756744,4.0,5,1,4


In [25]:
# Write dataframes into files
nodes_file = './Gephi/coordinates_nodes.csv'
edges_file = './Gephi/coordinates_edges.csv'

with open(nodes_file, 'w', encoding='utf-8') as outFile:
    nodes_df.to_csv(outFile, sep=',', encoding='utf-8', index=False)

with open(edges_file, 'w') as outFile:
    edges_df.to_csv(outFile, sep=',', encoding='utf-8', index=False)